**Kaggle** это самый крупный портал на котором проходят соревнования по машинному обучению
(участвовать в них может любой). В качестве домашнего задания предлагается попытать свои силы
на наборе данных **Титаник**.

Начните выполнение этого задания с посещения этой страницы:

https://www.kaggle.com/c/titanic

1. Перейдя по этой ссылке вы попадете на вкладку **Overview**, в верхнем правом углу есть кнопка **Join competition**, после того как вы ее нажмете вы сможете скачать данные по выжившим в крушении Титаника в разделе **Data**.

2. Там же в разделе **Data** в подразделе **Data description**, расположено описание всех признаков содержащихся в данных. 

3. Раздел **Kernels** содержит готовые решения, опубликованные разными людьми ввиде jupyter notebook'ов (некоторые на языке R, большая часть на языке Python).

4. Раздел **Discussion** - это форум посвященный данному соревнованию (Соревнование Titanic это учебное соревнование для всех).

5. В разделе **Leaderboard** можно увидеть результаты других участников соревнования (на тестовой выборке). Не удивляйтесь большому количеству *идеальных* accuracy (categorization accuracy это тоже самое что и обычное accuracy), данные о пассажирах Титаника есть в открытом доступе и некоторые зачем то их сабмитят (время от времени админы подтирают такие строчки). 

-----------------------


### Ваша ультимативная цель: провести анализ данных, обучить модель (подобрав параметры на кросс валидации), сделать предсказания на тестовом наборе данных и засабмитить их на Leaderboard что бы увидеть получившуюся accuracy.

Простой анализ данных и несложное решение для начинающих можно увидеть вот здесь https://www.kaggle.com/dejavu23/titanic-eda-to-ml-beginner
(оно было опубликовано буквально несколько часов назад).
По завершении выполнения домашки, сходите по этой ссылке и сравните свой jupyter notebook и этот.

----------------------

В этом ноутбуке (в котором вы сейчас ^_^) я проведу вас по основным этапам, но выполнить вы их должны будете самостоятельно.

Итак наметим план:

1. Скачаем данные: два .csv файла train_data и test_data, в файле test_data отсутствует колонка Survived (на них мы будем делать предсказания обученной моделью).
2. Откроем набор данных train_data в Jupyter notebook.
3. Проведем Обзорный Анализ Данных (Exploratory Data Analysis сокращенно EDA)
4. Обработаем категориальные признаки
5. Обработаем пропуски в данных (если они есть)
6. Обработаем числовые признаки
7. Обучим 3 модели (или больше):
    - Метод ближайшего соседа (подберем параметр k - число соседей, на кросс-валидации)
    - Метод логистической регрессии (подберем параметры регуляризации - alpha, на кросс-валидации)
    - Метод решающего дерева (подберем параметры глубины дерева, и другие на кросс-валидации)
8. Подобрав оптимальный набор параметров обучим наши модели на всех объектах из файла train_data.
9. Откроем набор данных test_data в Jupyter notebook.
10. Проведем шаги 4-6 над test_data аналогично тому как мы это делали на train_data. 
11. Сделаем предсказания с помощью моделей (обученных на шаге 7.) на обработанных test_data.
12. Сохраним предсказания в нужном формате и загрузим на Leaderboard на Kaggle.


На всякий случай помните, что в этой домашке главными являются пункты 3-11 если у вас возникнут трудности со скачиванием данных или загрузкой результатов предсказания на Leadearboard смело пишите мне я помогу (да и вообще в случае любых вопросов пишите).

Начнем

# 1. Скачаем данные

![data_download](download.png)

In [64]:
import io
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import Lasso, Ridge, SGDRegressor, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import make_regression

from sklearn.metrics import mean_squared_error

In [58]:
import requests

In [56]:
from sklearn.neighbors import KNeighborsClassifier     #KNN
from sklearn.linear_model import LogisticRegression    #Logistic Regression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
import sklearn.preprocessing

In [0]:
pd.options.mode.chained_assignment = None

In [59]:
train = pd.read_csv('/media/care1e55/windows/Users/care1e55/Downloads/titanic/train.csv') 

In [60]:
test = pd.read_csv('/media/care1e55/windows/Users/care1e55/Downloads/titanic/test.csv') 

In [65]:
X = train.drop(columns=['Survived'], inplace=False)
y = train['Survived']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [67]:
class Ttransoform(sklearn.preprocessing.FunctionTransformer):

    # def __init__(self, *featurizers):
        # self.featurizers = featurizers

    def fit(self, _train, y=None):
        
        # self.y = _train.drop(columns=['Survived'])
        # self._train = _train[['Survived']]

        return self

    def transform(self, _train):
        _train['Age'].fillna((_train['Age'].median()), inplace=True)
        _train['Fare'].fillna((_train['Fare'].median()), inplace=True)
        _train = _train.drop(columns=['Cabin'])
        _train = _train.drop(columns=['Name'])
        _train = _train.drop(columns=['Ticket'])

        # _train = _train.dropna()
        _train['Embarked'].fillna('S', inplace=True)
        # Get one hot encoding
        one_hot = pd.get_dummies(_train['Sex'])
        # Drop column
        _train = _train.drop('Sex',axis = 1)
        # Join the encoded df
        _train = _train.join(one_hot)

        # Get one hot
        one_hot = pd.get_dummies(_train['Embarked'])
        # Drop column
        _train = _train.drop('Embarked',axis = 1)
        # Join the encoded df
        _train = _train.join(one_hot)
        
        return _train

In [68]:
transformator = Ttransoform()

In [71]:
tree_params = {
    'max_depth': 3,
    'min_samples_leaf': 7,
    'max_leaf_nodes': 20,
    'random_state': 3
}

dtc = DecisionTreeClassifier(**tree_params)
pipe = Pipeline([ ('transf', transformator), ('dtc', dtc)])
dic = {'dtc__max_features': [1,2,3,4,5,6,7,8,9,10], 'dtc__max_depth': [5,6,7,8,9,10]}
param_grid = {'max_depth': np.arange(3, 10)}
dtc_ = GridSearchCV(pipe, dic, cv=5, verbose=1).fit(X, y.values.ravel())
dtc_.best_estimator_.get_params()


# dtc_ = GridSearchCV(DecisionTreeClassifier(), dic, cv=5, verbose=1).fit(X, y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    7.6s finished


{'memory': None,
 'steps': [('transf',
   Ttransoform(accept_sparse=False, check_inverse=True, func=None,
         inv_kw_args=None, inverse_func=None, kw_args=None,
         pass_y='deprecated', validate=None)),
  ('dtc',
   DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
               max_features=8, max_leaf_nodes=20, min_impurity_decrease=0.0,
               min_impurity_split=None, min_samples_leaf=7,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               presort=False, random_state=3, splitter='best'))],
 'transf': Ttransoform(accept_sparse=False, check_inverse=True, func=None,
       inv_kw_args=None, inverse_func=None, kw_args=None,
       pass_y='deprecated', validate=None),
 'dtc': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
             max_features=8, max_leaf_nodes=20, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=7,
             min_samples_split=2, min_wei

In [73]:
dtc_.score(X_test, y_test)

0.8694029850746269

In [74]:
transformator.transform(test)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1
5,897,3,14.0,0,0,9.2250,0,1,0,0,1
6,898,3,30.0,0,0,7.6292,1,0,0,1,0
7,899,2,26.0,1,1,29.0000,0,1,0,0,1
8,900,3,18.0,0,0,7.2292,1,0,1,0,0
9,901,3,21.0,2,0,24.1500,0,1,0,0,1


In [75]:
__dtc =   DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
             max_features=8, max_leaf_nodes=20, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=7,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             presort=False, random_state=3, splitter='best')
__dtc.fit(transformator.transform(X), y.values.ravel())     
predictions = __dtc.predict(transformator.transform(test))

In [76]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [77]:
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predictions})
filename = 'Titanic Predictions 1.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: Titanic Predictions 1.csv


In [337]:
pd.options.mode.chained_assignment = None

In [348]:
print(X.shape)
print(y.values.ravel().shape)

(891, 12)
(891,)


In [349]:
y.values.ravel()

array([0, 1, ..., 1, 0])

In [347]:
knn = KNeighborsClassifier(n_neighbors=4, metric='euclidean', n_jobs=-1)
pipe = Pipeline([ ('transf', transformator), ('knn', knn)])
search_space = [{'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]
knn_ = GridSearchCV(pipe, search_space, cv=5, verbose=0).fit(X, y.values.ravel())
knn_.best_estimator_.get_params()

/home/care1e55/progz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


''

In [350]:
knn_.score(X_test, y_test)

0.7052238805970149